Početak 2020-02-18

In [1]:
import pandas as pd
from pandas import DataFrame as df
from google.colab import files
import datetime

In [2]:
start_date="2021-05-20"
end_date="2021-05-21"
wunderground_station_id="IPTUJ11"
station_id="DUDU1234"

range_of_dates=pd.date_range(start=start_date, end=end_date)
range_of_dates=list(map(lambda x: str(x)[:10],range_of_dates))

In [5]:
for measurement_date in range_of_dates:

  url=f"https://www.wunderground.com/dashboard/pws/{wunderground_station_id}/table/{measurement_date}/{measurement_date}/daily"
  dfs = pd.read_html(url,match="Precip. Rate.")
  data = pd.concat(dfs)
  data.dropna(inplace = True)
  data.to_csv(f"{wunderground_station_id}_{measurement_date}")

In [6]:
data=data.replace(to_replace=r'°F', value='', regex=True)
data=data.replace(to_replace=r'°%', value='', regex=True)
data=data.replace(to_replace=r'°in', value='', regex=True)
data=data[['Time', 'Temperature', 'Dew Point', 'Humidity', 'Pressure', 'Precip. Accum.']]
data['Temperature']=data['Temperature'].map(lambda x: round((((float(x)-32) * 5)/9),1))
data['Dew Point']=data['Dew Point'].map(lambda x: round((((float(x)-32) * 5)/9),1))
data['Pressure']=data['Pressure'].map(lambda x: round((float(x)*33.8639),1))
data['Precip. Accum.']=data['Precip. Accum.'].map(lambda x: round((float(x)*25.4),1))
data['Time'] = (pd.to_datetime(measurement_date+ " " + data['Time'], format='%Y-%m-%d %I:%M %p')-datetime.timedelta(hours=2)).dt.strftime('%Y%m%d%H%M')
data

,Time,Temperature,Dew Point,Humidity,Pressure,Precip. Accum.
1,202105202203,9.6,5.7,77,1013.5,0.0
2,202105202214,8.9,5.9,81,1013.9,0.0
3,202105202215,8.9,5.5,79,1013.9,0.0
4,202105202224,8.9,5.9,81,1013.9,0.0
5,202105202227,8.7,5.8,82,1013.5,0.0
...,...,...,...,...,...,...
222,202105212131,15.8,6.4,54,1005.4,0.0
223,202105212139,15.7,6.4,54,1005.4,0.0
224,202105212143,15.8,6.2,53,1005.4,0.0
225,202105212154,16.0,6.4,53,1005.4,0.0


#Precipitation

In [9]:
daily_precipitation=data['Precip. Accum.'].sum()
if daily_precipitation < 0.5:
  daily_precipitation= 'T'

daily_precipitation

'T'

In [10]:
#2155 is time for summer period, and 2255 is time for winter period
f"PR {station_id} 1 {measurement_date.replace('-','')}2155 {daily_precipitation} 1 X X"

'PR DUDU1234 1 202105212155 T 1 X X'

#Temperature

In [ ]:
for measurement_time, temperature in zip(data['Time'], data['Temperature']):
    print(f"ATMNN {station_id} 01 {measurement_time} {temperature}")

#Pressure

In [ ]:
for measurement_time, pressure in zip(data['Time'], data['Pressure']):
    print(f"BP {station_id} 01 {measurement_time} 1 {pressure}")

#Humidity

In [16]:


for measurement_time, temperature, humidity in zip(data['Time'], data['Temperature'], data['Humidity']):
    print(f"RH {station_id} 01 {measurement_time} D {temperature} X {humidity}")
#RH ZZZZ1234 1 200011260101 D 28.5 X 80

RH DUDU1234 01 202105202203 D 9.6 X 77 
RH DUDU1234 01 202105202214 D 8.9 X 81 
RH DUDU1234 01 202105202215 D 8.9 X 79 
RH DUDU1234 01 202105202224 D 8.9 X 81 
RH DUDU1234 01 202105202227 D 8.7 X 82 
RH DUDU1234 01 202105202249 D 8.3 X 86 
RH DUDU1234 01 202105202251 D 8.3 X 86 
RH DUDU1234 01 202105202259 D 8.1 X 86 
RH DUDU1234 01 202105202303 D 7.8 X 87 
RH DUDU1234 01 202105202314 D 7.8 X 87 
RH DUDU1234 01 202105202315 D 7.8 X 87 
RH DUDU1234 01 202105202324 D 8.1 X 85 
RH DUDU1234 01 202105202327 D 8.1 X 85 
RH DUDU1234 01 202105202339 D 8.0 X 83 
RH DUDU1234 01 202105202349 D 7.6 X 86 
RH DUDU1234 01 202105202351 D 7.4 X 86 
RH DUDU1234 01 202105202359 D 7.2 X 88 
RH DUDU1234 01 202105210003 D 7.2 X 88 
RH DUDU1234 01 202105210014 D 7.0 X 89 
RH DUDU1234 01 202105210015 D 7.0 X 89 
RH DUDU1234 01 202105210024 D 7.1 X 87 
RH DUDU1234 01 202105210039 D 6.9 X 88 
RH DUDU1234 01 202105210049 D 6.7 X 89 
RH DUDU1234 01 202105210051 D 6.7 X 90 
RH DUDU1234 01 202105210103 D 6.8 X 90 


#Other

In [ ]:
files.download()